In [24]:
from pydrake.parsers import PackageMap
from pydrake import getDrakePath
import os.path
import scipy.io
import numpy as np
import pydrake.rbtree as rbt
import pydrake.solvers.ik as ik
from pydrake.trajectories import PiecewisePolynomial
from pydrake.lcm import DrakeLcm
from pydrake.systems import DrakeVisualizer

In [25]:
fp = scipy.io.loadmat(os.path.join(getDrakePath(), "examples", "Atlas", "data", "atlas_fp.mat"))

In [28]:
xstar = fp["xstar"]

In [2]:
pm = PackageMap()
model = os.path.join(getDrakePath(), "examples", "Atlas", "urdf", "atlas_minimal_contact.urdf")
pm.PopulateUpstreamToDrake(model)
robot = rbt.RigidBodyTree(model, package_map=pm, floating_base_type=rbt.FloatingBaseType.kRollPitchYaw)

In [29]:
constraints = [
    ik.WorldPositionConstraint(robot,
                               robot.findFrame("r_foot_sole").get_frame_index(),
                               np.array([0., 0, 0]),
                               np.array([0, -0.5, 0]),
                               np.array([0, -0.5, 0])
                              ),
    ik.WorldPositionConstraint(robot,
                               robot.findFrame("l_foot_sole").get_frame_index(),
                               np.array([0., 0, 0]),
                               np.array([0., 0, 0]),
                               np.array([0., 0, 0])
                              ),
    ik.WorldCoMConstraint(robot,
                          np.array([0, -0.25, 1]),
                          np.array([0, -0.25, 1])
                         )
]
q_seed = xstar[:robot.number_of_positions()]
options = ik.IKoptions(robot)
results = ik.InverseKin(robot, q_seed, q_seed, constraints, options)

x = np.pad(results.q_sol[0], (0, robot.number_of_velocities()), "constant")

pp = PiecewisePolynomial.ZeroOrderHold([0., 0.001], [x, x])

lcm = DrakeLcm()

vis = DrakeVisualizer(robot, lcm)

vis.PlaybackTrajectory(pp)